В рамках этого итогового задания мы будем прогнозировать сердечную недостаточность.
https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction

оберните весь конвейер преобразований в Pipeline

подберите оптимальный вариант прогнозной модели с помощью GridSearchCV

примените обученный на тренировочных данных конвейер к тестовым данным, никак не предобрабатывая их, а лишь загрузив из файла и отделив целевой признак от остальных

получите на тестовой части качество не ниже 0.87 по метрике ROCAUC

In [1]:
import pandas as pd
import dill as pickle
import requests
import json
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

pd.set_option('display.max_columns', 50)

Подгружаем данные и изучаем их 

In [2]:
data_train = pd.read_csv('heart_adapt_train.csv')
data_train.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,74.0,M,NAP,138.0,NaN,0,Normal,116,N,0.2,Up,0
1,58.0,M,NAP,132.0,224.0,0,LVH,173,N,3.2,Up,1
2,44.0,M,ATA,150.0,288.0,0,Normal,150,Y,3.0,Flat,1
3,50.0,M,ASY,144.0,349.0,0,LVH,120,Y,1.0,Up,1
4,NaN,M,ASY,145.0,248.0,0,Normal,96,Y,2.0,Flat,1


In [3]:
data_test = pd.read_csv('heart_adapt_test.csv')
data_test.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,44.0,M,NAP,130.0,233.0,0,Normal,179,Y,0.4,Up,0
1,63.0,M,ASY,130.0,308.0,0,Normal,138,Y,2.0,Flat,1
2,35.0,F,TA,120.0,160.0,0,ST,185,N,0.0,Up,0
3,69.0,M,NAP,140.0,NaN,1,ST,118,N,2.5,Down,1
4,NaN,M,TA,142.0,200.0,1,ST,100,N,1.5,Down,1


In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             533 non-null    float64
 1   Sex             589 non-null    object 
 2   ChestPainType   589 non-null    object 
 3   RestingBP       588 non-null    float64
 4   Cholesterol     462 non-null    float64
 5   FastingBS       589 non-null    int64  
 6   RestingECG      589 non-null    object 
 7   MaxHR           589 non-null    int64  
 8   ExerciseAngina  589 non-null    object 
 9   Oldpeak         589 non-null    float64
 10  ST_Slope        589 non-null    object 
 11  HeartDisease    589 non-null    int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 55.3+ KB


In [5]:
data_train['Age'].fillna(data_train['Age'].median(), inplace=True)
data_train['Cholesterol'].fillna(data_train['Cholesterol'].median(), inplace=True)
data_train['RestingBP'].fillna(data_train['RestingBP'].median(), inplace=True)

Отделяем признаки от целевой переменной.

In [6]:
features_train = data_train.drop(['HeartDisease'], axis=1).copy()
target_train = data_train['HeartDisease'].copy()

In [7]:
features_test = data_test.drop(['HeartDisease'], axis=1).copy()
target_test = data_test['HeartDisease'].copy()

Классы сбалансированны?

In [8]:
target_train.value_counts()

1    381
0    208
Name: HeartDisease, dtype: int64

Посмотрим на значения категориальных переменных

In [9]:
features_train['Sex'].unique()

array(['M', 'F'], dtype=object)

In [10]:
features_train['ChestPainType'].unique()

array(['NAP', 'ATA', 'ASY', 'TA'], dtype=object)

In [11]:
features_train['RestingECG'].unique()

array(['Normal', 'LVH', 'ST'], dtype=object)

In [12]:
features_train['ExerciseAngina'].unique()

array(['N', 'Y'], dtype=object)

In [13]:
features_train['ST_Slope'].unique()

array(['Up', 'Flat', 'Down'], dtype=object)

Обработаем пропущенные данные

Изменим типы столбцов

In [14]:
class HeartTypesToFlags(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None): 
        self.types_ChestPainType = sorted(set(X['ChestPainType']))
        self.types_RestingECG = sorted(set(X['RestingECG']))
        self.types_ST_Slope = sorted(set(X['ST_Slope']))
        return self

    def transform(self, X):
        X = X.reset_index(drop=True)
        
        # Бинарное кодирование 'Sex' и 'ExerciseAngina'
        X['Sex'] = (X['Sex'] == 'M').astype(int)
        X['ExerciseAngina'] = (X['ExerciseAngina'] == 'Y').astype(int)
        
        # Применение one-hot encoding к каждому категориальному признаку
        ohe_types_ChestPainType = pd.get_dummies(X['ChestPainType'])
        ohe_types_RestingECG = pd.get_dummies(X['RestingECG'])
        ohe_types_ST_Slope = pd.get_dummies(X['ST_Slope'])
        
        # Соединение всех результатов
        X = pd.concat([X.drop(['ChestPainType', 'RestingECG', 'ST_Slope'], axis=1),
                       ohe_types_ChestPainType, ohe_types_RestingECG, ohe_types_ST_Slope], axis=1)
        return X


Преобразуем тренировочный датасет 

In [15]:
httf = HeartTypesToFlags()
httf.fit(features_train)
features_train_httf = httf.transform(features_train)
features_train_httf.head()

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ASY,ATA,NAP,TA,LVH,Normal,ST,Down,Flat,Up
0,74.0,1,138.0,240.0,0,116,0,0.2,0,0,1,0,0,1,0,0,0,1
1,58.0,1,132.0,224.0,0,173,0,3.2,0,0,1,0,1,0,0,0,0,1
2,44.0,1,150.0,288.0,0,150,1,3.0,0,1,0,0,0,1,0,0,1,0
3,50.0,1,144.0,349.0,0,120,1,1.0,1,0,0,0,1,0,0,0,0,1
4,55.0,1,145.0,248.0,0,96,1,2.0,1,0,0,0,0,1,0,0,1,0


Отмасштабируем признаки

In [16]:
scaler = StandardScaler()
scaler.fit(features_train_httf)
features_train_httf_scaled = pd.DataFrame(scaler.transform(features_train_httf), columns=scaler.feature_names_in_)
features_train_httf_scaled.head()

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ASY,ATA,NAP,TA,LVH,Normal,ST,Down,Flat,Up
0,2.176863,0.484563,0.246906,-0.085128,-0.589768,-0.758105,-0.901281,-0.689009,-1.201659,-0.441278,2.008529,-0.223407,-0.521669,0.840979,-0.500531,-0.269925,-1.109532,1.276505
1,0.411393,0.484563,-0.071898,-0.393449,-0.589768,1.529085,-0.901281,2.113660,-1.201659,-0.441278,2.008529,-0.223407,1.916925,-1.189090,-0.500531,-0.269925,-1.109532,1.276505
2,-1.133393,0.484563,0.884515,0.839833,-0.589768,0.606184,1.109532,1.926815,-1.201659,2.266146,-0.497877,-0.223407,-0.521669,0.840979,-0.500531,-0.269925,0.901281,-0.783389
3,-0.471341,0.484563,0.565711,2.015306,-0.589768,-0.597600,1.109532,0.058369,0.832183,-0.441278,-0.497877,-0.223407,1.916925,-1.189090,-0.500531,-0.269925,-1.109532,1.276505
4,0.080368,0.484563,0.618845,0.069032,-0.589768,-1.560627,1.109532,0.992592,0.832183,-0.441278,-0.497877,-0.223407,-0.521669,0.840979,-0.500531,-0.269925,0.901281,-0.783389


Построим простой классификатор с помощью полученных признаков и оценим его качество

In [17]:
model = DecisionTreeClassifier(random_state=0, class_weight='balanced')
model.fit(features_train_httf_scaled, target_train)
print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, model.predict_proba(features_train_httf_scaled)[:, 1])})

Качество модели на обучающей выборке: {1.0}


Подготовим конвейнер, чтобы можно было запустить все эти трансформеры последовательно одной командой

In [18]:
pipe = Pipeline([
    ('ohe_types', HeartTypesToFlags()),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classify', DecisionTreeClassifier(random_state=0, class_weight='balanced'))
])

Настраиваем трансформеры однократно на исходной обучающей выборке и проверяем, что они работают, как ожидается.

In [19]:
pipe.fit(X=features_train, y=target_train)
print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, pipe.predict_proba(features_train)[:, 1])})

Качество модели на обучающей выборке: {1.0}


Проверим качество на тестовой части

In [20]:
print('Качество модели на тестовой выборке:', {roc_auc_score(target_test, pipe.predict_proba(features_test)[:, 1])})

Качество модели на тестовой выборке: {0.8037682789651294}


In [21]:
pipe['classify'].tree_.max_depth

14

Переберем несколько вариантов, с помощью кросс-валидации, чтобы улучшить качество на тестовой выборке

In [22]:
pipe = Pipeline([  
    ('ohe_types', HeartTypesToFlags()),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classify', DecisionTreeClassifier(random_state=0, class_weight='balanced'))
])

params = [
    {'classify': [LogisticRegression(class_weight='balanced', random_state=0)]}, 
    {'classify': [DecisionTreeClassifier(class_weight='balanced', random_state=0)], 'classify__max_depth': [2, 5, 10, 20]},
    {'classify': [RandomForestClassifier(class_weight='balanced', random_state=0)], 'classify__n_estimators': [50, 100, 200]},
    {'classify': [GradientBoostingClassifier(random_state=0)], 'classify__n_estimators': [50, 100, 200], 'classify__learning_rate': [0.01, 0.1, 0.2]}
]

grid_search = GridSearchCV(pipe, param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X=features_train, y=target_train)
print(
    'Качество модели на тестовой выборке c лучшей моделью:', 
    {roc_auc_score(target_test, grid_search.predict_proba(features_test)[:, 1])}
)

Качество модели на тестовой выборке c лучшей моделью: {0.9098425196850393}


In [23]:
grid_search.best_estimator_

Pipeline(steps=[('ohe_types', HeartTypesToFlags()),
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler()),
                ('classify',
                 RandomForestClassifier(class_weight='balanced',
                                        n_estimators=50, random_state=0))])

### Оборачивание модели в сервис API

Сериализуем (консервируем) модель

In [24]:
with open('best_heart_disease_model.pk', 'wb') as file:
    pickle.dump(grid_search, file)

Десериализуем ее, чтобы убедиться, что она правильно работает

In [25]:
with open('best_heart_disease_model.pk','rb') as f:
    loaded_model = pickle.load(f)

In [26]:
print(
    'Качество модели на тестовой выборке от законсервированной модели:', 
    {roc_auc_score(target_test, loaded_model.predict_proba(features_test)[:, 1])}
)

Качество модели на тестовой выборке от законсервированной модели: {0.9098425196850393}


Функцию-обертку реализуем в отдельном файле best_heart_disease_service.py

Чтобы запустить API, нужно в терминале перейти в папку с кодом сервиса и ввести ```waitress-serve --host=0.0.0.0 --port=8000 best_heart_disease_service:app```  
Если ошибка ```Connection in use: ('0.0.0.0', 8000)```
- то либо делаем ```kill <номер, который у процесса, который занял порт>```
- либо пробуем вместо 8000 другой

После запуска API, можно им пользоваться 

In [37]:
features_test

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,44.0,M,NAP,130.0,233.0,0,Normal,179,Y,0.4,Up
1,63.0,M,ASY,130.0,308.0,0,Normal,138,Y,2.0,Flat
2,35.0,F,TA,120.0,160.0,0,ST,185,N,0.0,Up
3,69.0,M,NAP,140.0,NaN,1,ST,118,N,2.5,Down
4,NaN,M,TA,142.0,200.0,1,ST,100,N,1.5,Down
...,...,...,...,...,...,...,...,...,...,...,...
192,38.0,M,ATA,140.0,297.0,0,Normal,150,N,0.0,Up
193,63.0,M,TA,145.0,233.0,1,LVH,150,N,2.3,Down
194,51.0,F,NAP,140.0,308.0,0,LVH,142,N,1.5,Up
195,53.0,M,ASY,125.0,NaN,1,Normal,120,N,1.5,Up


In [34]:
header = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

resp = requests.post(
    "http://127.0.0.1:8000/predict", 
    data=json.dumps(features_test.to_json(orient='records')),
    headers=header
)

In [35]:
resp.status_code

200

In [36]:
print(
    'Качество модели на тестовой выборке от модели в API:', 
    {roc_auc_score(target_test, pd.read_json(resp.json()['predictions']))}
)

Качество модели на тестовой выборке от модели в API: {0.9098425196850393}
